In [2]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

In [3]:
# movies - index, movie name, catagory
movies = pd.read_csv('ml-1m/movies.dat', sep='::', header=None, engine='python', encoding='latin-1')
# users - index, gender, age, financial info, zip code
users = pd.read_csv('ml-1m/users.dat', sep='::', header=None, engine='python', encoding='latin-1')
# ratings - index, user id, movie id, rating (1-5), timestaps (not required)
ratings = pd.read_csv('ml-1m/ratings.dat', sep='::', header=None, engine='python', encoding='latin-1')
# training set - index, user index, movie index, rating (1-5), time stamp (not required)
training_set = pd.read_csv('ml-100k/u1.base', delimiter='\t')
training_set = np.array(training_set, dtype='int')
test_set = pd.read_csv('ml-100k/u1.test', delimiter='\t')
test_set = np.array(test_set, dtype='int')

# Getting the number of users and movies
# taking max of max as the value can be in either train / test set
nb_users = int(max(max(training_set[:, 0]), max(test_set[:, 0])))
nb_movies = int(max(max(training_set[:, 1]), max(test_set[:, 1])))

# converting the data into an array with users in lines and movies in columns
def convert(data):
    new_data = []
    for id_users in range(1, nb_users + 1):
        id_movies = data[:, 1][data[:, 0] == id_users]
        id_ratings = data[:, 2][data[:, 0] == id_users]
        ratings = np.zeros(nb_movies) # this also adds a 0 for all the movies not rated by the user
        ratings[id_movies - 1] = id_ratings 
        new_data.append(list(ratings)) # list of lists
    return new_data

training_set = convert(training_set)
test_set = convert(test_set)

# converting the data into torch tensors
# floatTensor class expects a list of lists
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

# # Converting ratings into binary ratings, 1 (liked) & 0 (Not liked)
# training_set[training_set == 0] = -1 # these were the movies not rated by the user.
# training_set[training_set == 1] = 0
# training_set[training_set == 2] = 0 # torch doesn't accept 'or' operator, hence they have been written separately
# training_set[training_set >= 3] = 1

# test_set[test_set == 0] = -1 # these were the movies not rated by the user.
# test_set[test_set == 1] = 0
# test_set[test_set == 2] = 0 # torch doesn't accept 'or' operator, hence they have been written separately
# test_set[test_set >= 3] = 1

In [8]:
class SAE(nn.Module):
    def __init__(self, ): # This function creates the architecture.
        super(SAE, self).__init__()
        self.fc1 = nn.Linear(nb_movies, 20) # 20 nodes in the 1st hidden layers.
        self.fc2 = nn.Linear(20, 10) # 10 neurons in the 2nd hidden layer.
        # next layer will start reconstructing, hence the architecture will be made symmetric.
        self.fc3 = nn.Linear(10, 20) #30 neurons in the 3rd hidden layer.
        self.fc4 = nn.Linear(20, nb_movies) # no of neurons in O/P layer must be same as that of I/P layer.
        self.activation = nn.Sigmoid()

    # This function performs the action (forward propagation).
    def forward(self, x): # x - input vector.
        # 1st encoding (I/P vector is compressed into a shorter version).
        x = self.activation(self.fc1(x))
        x = self.activation(self.fc2(x))
        x = self.activation(self.fc3(x))
        x = self.fc4(x) 
        return x

In [17]:
sae = SAE()
criterion = nn.MSELoss()
optimizer = optim.RMSprop(sae.parameters(), lr=0.01, weight_decay=0.5)
nb_epoch = 20

In [18]:
for epoch in range(1, nb_epoch + 1):
    train_loss = 0  # loss is 0 at initial iteration.
    s = 0.  # initializing s (rmse) as a float.
    # this loop will loop over all observations in each epoch.
    for id_users in range(nb_users):
        input = Variable(training_set[id_users]).unsqueeze(0)  # creating a batch of one input vector.
        target = input.clone()  # right now the target and input are the same, will be modified later.
        # Next step will exclude all users who haven't rated any movie.
        if torch.sum(target.data > 0) > 0:
            output = sae(input)
            target.requires_grad = False  # don't compute gradient w.r.t the target (saves computation).
            output[target == 0] = 0  # these values will not be considered for error.
            loss = criterion(output, input)
            mean_corrector = nb_movies / float(torch.sum(target.data > 0) + 1e-10)
            loss.backward()  # decides the direction of weight update.
            train_loss += np.sqrt(loss.item() * mean_corrector)  # Use `.item()` to extract scalar.
            s += 1.
            optimizer.step()  # decides the intensity of weight updates.
    print('Epoch: ' + str(epoch) + ' loss: ' + str(train_loss / s))


Epoch: 1 loss: 1.7717441699028045
Epoch: 2 loss: 1.0966161820831701
Epoch: 3 loss: 1.0533194063545395
Epoch: 4 loss: 1.038493246080098
Epoch: 5 loss: 1.0307201224783613
Epoch: 6 loss: 1.02659210412905
Epoch: 7 loss: 1.0236172656512
Epoch: 8 loss: 1.0217308318431106
Epoch: 9 loss: 1.020608061446971
Epoch: 10 loss: 1.019314434155422
Epoch: 11 loss: 1.018610481179131
Epoch: 12 loss: 1.0185385382748682
Epoch: 13 loss: 1.0176785763154506
Epoch: 14 loss: 1.0176363222112716
Epoch: 15 loss: 1.0169759076208433
Epoch: 16 loss: 1.0169206086162124
Epoch: 17 loss: 1.0165547265055423
Epoch: 18 loss: 1.0163373613378415
Epoch: 19 loss: 1.016164696403952
Epoch: 20 loss: 1.016210595463455


In [20]:
# Testing the SAE.
test_loss = 0  # Initialize test loss.
s = 0.  # Initialize the count for normalization.
for id_users in range(nb_users):
    input = Variable(training_set[id_users]).unsqueeze(0)  # Reshape input to [1, 1682].
    target = Variable(test_set[id_users]).unsqueeze(0)  # Reshape target to [1, 1682].
    if torch.sum(target.data > 0) > 0:
        output = sae(input)
        target.requires_grad = False  # Prevent gradient computation for target.
        output[target == 0] = 0  # Mask values where target is 0.
        loss = criterion(output, target)  # Compute loss against the target.
        mean_corrector = nb_movies / float(torch.sum(target.data > 0) + 1e-10)  # Correct loss for rated movies.
        test_loss += np.sqrt(loss.item() * mean_corrector)  # Use `.item()` to extract scalar loss.
        s += 1.  # Increment count for averaging.
print('Test loss: ' + str(test_loss / s))  # Print normalized test loss.


Test loss: 1.0229217496252785
